In [1]:
from gurobipy import GRB
import gurobipy as gb

Objective: minimizes the total amount that Sunnyshore Bay has to repay to the bank over the summer months.

Define Variables: 

X i : Amount borrowed in month i

Y i : Amount repaid in month i

W i : Cash balance at the end of month i


Write the mathematical objective function:

Minimize Z = (Xi+Interest of Xi - Yi) = (X5 + )


In [ ]:
# Create the optimization model
model = gb.Model("Sunnyshore Bay")

IR = [1 + 1.75/100, 1 + 2.25/100, 1 + 2.75/100]

# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
B1 = model.addVars(3, lb=0, vtype=GRB.CONTINUOUS, name="Borrow_May")
B2 = model.addVars(2, lb=0, vtype=GRB.CONTINUOUS, name="Borrow_Jun")
B3 = model.addVars(1, lb=0, vtype=GRB.CONTINUOUS, name="Borrow_Jul")

# auxiliary decision variables (cash balance at the end of period)
w = model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Wealth")

# The objective function
Loan_may = gb.quicksum(B1[i]*IR[i] for i in range(3))
Loan_jun = gb.quicksum(B2[i]*IR[i] for i in range(2))
Loan_jul = B3[0]*IR[0]
model.setObjective(Loan_may + Loan_jun + Loan_jul, GRB.MINIMIZE)

# Add the constraints
model.addConstr(w[0] == 140000 + 180000 + B1[0] + B1[1] + B1[2] - 300000, "May Constraint")
model.addConstr(w[1] == w[0] + B2[0] + B2[1] + 260000 - IR[0]*B1[0] - 400000, "Jun Constraint")
model.addConstr(w[2] == w[1] + B3[0] + 420000 - 350000 - IR[1]*B1[1] - B2[0]*IR[0], "Jul Constraint")
model.addConstr(w[3] == w[2] + 580000 - 200000 - B1[2]*IR[2] - B2[1]*IR[1] - B3[0]*IR[0], "Aug Constraint")

# We could also define these constraints as upper bounds in the definition of the decision variables
model.addConstr(B1[0] + B1[1] + B1[2] <= 250000, "Borrowing Constraint May")
model.addConstr(B2[0] + B2[1] <= 150000, "Borrowing Constraint Jun")
model.addConstr(B3[0] <= 350000, "Borrowing Constraint May")

# Cash balance constraint
cash_may = model.addConstr(w[0] >= 25000, "Cash Constraint May")
cash_jun = model.addConstr(w[1] >= 20000, "Cash Constraint Jun")
cash_jul = model.addConstr(w[2] >= 35000, "Cash Constraint Jul")
cash_aug = model.addConstr(w[3] >= 18000, "Cash Constraint Aug")
cash_ratio = model.addConstr(w[2] >= (w[0] + w[1])*0.65, "Cash ratio constraint")

# Optimally solve the problem
model.optimize()

# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# Number of constraints in the model
print("Number of Constraints: ", model.numConstrs)

# The status of the model
print("Model Status: ", model.status)

# Value of the objective function
print("Loan repayment:", model.objVal)

# Print the decision variables
print(model.printAttr('X'))

# Print sensitivity information
print("")
print(f"Sensitivity Information for Jun cash {cash_jun.pi:.2f}:")
print("(LHS, RHS, Slack): ", (model.getRow(cash_jun).getValue(), cash_jun.RHS, cash_jun.slack))
print("Shadow Price: ", cash_jun.pi)
print("Range of Feasibility: ", (cash_jun.SARHSUp, cash_jun.SARHSLow))

print("Additional repayment due to $7.5k increase in Jun cash: ", cash_jun.pi*7500)
print("Total repayment after: ", cash_jun.pi*7500 + model.objVal)

print("")
print("List of Shawdow Price")
for constr in model.getConstrs():
    print(f"{constr.ConstrName}: {constr.Pi}")

In [ ]:
import numpy as np

# Create the optimization model
model = gb.Model("Sunnyshore Bay Dual")

# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
y = model.addVars(3, lb=0, vtype=GRB.CONTINUOUS, name="Borrowing dual variables")
z = model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Cash balance dual variables")
w = model.addVars(4, vtype=GRB.CONTINUOUS, name="Cash equality dual variables")
v = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Cash ratio dual variables")

for i in range(4):
    w[i].VarName = f"w[{i}]"
    w[i].lb = -1e200  # Large negative value
    w[i].ub = 1e200   # Large positive value

# The objective function
c = [1.0175,1.0225,1.0275,1.0175,1.0225,1.0175,0,0,0,0]

RHS_y = [-250000,-150000,-350000]
RHS_z = [25000,20000,35000,18000]
RHS_w = [20000,-140000,70000,380000]
RHS_v = [0]

coeff_borrowing = [[-1,-1,-1,0,0,0,0,0,0,0],
                   [0,0,0,-1,-1,0,0,0,0,0],
                   [0,0,0,0,0,-1,0,0,0,0]]

coeff_cash_bal = [[0,0,0,0,0,0,1,0,0,0],
                  [0,0,0,0,0,0,0,1,0,0],
                  [0,0,0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,0,0,0,0,1]]

coeff_cash_eq =  [[-1,-1,-1,0,0,0,1,0,0,0],
                  [1.0175,0,0,-1,-1,0,-1,1,0,0],
                  [0,1.0225,0,1.0175,0,-1,0,-1,1,0],
                  [0,0,1.0275,0,1.0225,1.0175,0,0,-1,1]]

coeff_cash_ratio = [[0,0,0,0,0,0,-0.65,-0.65,1,0]]

model.setObjective(gb.quicksum(RHS_y[i]*y[i] for i in range(3)) + gb.quicksum(RHS_z[j]*z[j] for j in range(4))
                   + gb.quicksum(RHS_w[k]*w[k] for k in range(4)) + RHS_v[0]*v, GRB.MAXIMIZE)

# Add the constraints
coeff = np.transpose(np.vstack((coeff_borrowing, coeff_cash_bal, coeff_cash_eq, coeff_cash_ratio)))

for i in range(10):
    model.addConstr((gb.quicksum(y[j]*coeff[i,j] for j in range(3)) +
                        gb.quicksum(z[m]*coeff[i,m+3] for m in range(4)) +
                        gb.quicksum(w[n]*coeff[i,n+7] for n in range(4)) +
                        v*coeff[i,11] <= c[i]), "Dual Constraint")

# Optimally solve the problem
model.optimize()

# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# Number of constraints in the model
print("Number of Constraints: ", model.numConstrs)

# The status of the model
print("Model Status: ", model.status)

# Value of the objective function
print("Loan repayment:", model.objVal)

# Print the decision variables
print(model.printAttr('X'))

print("List of Shawdow Price")
for constr in model.getConstrs():
    print(f"{constr.ConstrName}: {constr.Pi}")

In [ ]:
# Define given data
months = ['May', 'June', 'July'] # No need to list 'August'
revenues = [180000, 260000, 420000, 580000]
expenses = [300000, 400000, 350000, 200000]
min_cash_balance = [25000, 20000, 35000, 18000]
max_borrow_amount = [250000, 150000, 350000]
init_cash_balance = 140000
interest_rates = {
    'May': [0.0175, 0.0225, 0.0275],
    'June': [0.0175, 0.0225],
    'July': [0.0175]
}

def find_model_optimization(model):
    # Decision variables
    borrow = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, name=f"borrow_{i}_{j}")
          for i in months for j in range(len(interest_rates[i]))}

    # Objective function: minimize total repayment
    model.setObjective(gb.quicksum(borrow[i, j] * interest_rates[i][j] for i in months for j in range(len(interest_rates[i]))),
                   GRB.MINIMIZE)

    # Constrints
    for i in range(len(months)):
        # cash balance constraints: borrow + init_balance + revenue - expenses >= min_cash_balance
        model.addConstr(gb.quicksum(borrow[months[i], j] for j in range(len(interest_rates[months[i]]))) +
                    init_cash_balance + revenues[i] - expenses[i] >= min_cash_balance[i], f"cash_balance_{months[i]}")

        # total amount borrowed constraint
        model.addConstr(gb.quicksum(borrow[months[i], j] for j in range(len(interest_rates[months[i]]))) <=
                    max_borrow_amount[i], f"total_borrowed_{months[i]}")


        # cash balance constraint for end of July
        may_cash = gb.quicksum(borrow[months[0], j] * (1 + interest_rates[months[0]][j]) for j in range(len(interest_rates[months[0]])))
        june_cash = gb.quicksum(borrow[months[1], j] * (1 + interest_rates[months[1]][j]) for j in range(len(interest_rates[months[1]])))
        july_cash = borrow[months[2], 0] * (1 + interest_rates[months[2]][0])

        model.addConstr(may_cash + june_cash + july_cash >= 0.65 *
                (init_cash_balance + revenues[0] + init_cash_balance + revenues[1]),
                "end_of_July_constraint")

    # optimize the Model
    model.optimize()

    # display the results
    if model.status == gb.GRB.OPTIMAL:
        print("Optimal solution found!")
        total_repayment = model.objVal
        print(f"Total repayment: ${total_repayment:.2f}")

        for i in months:
            for j in  range(len(interest_rates[i])):
                print(f"Borrow in {i} at rate {interest_rates[i][j]}: ${borrow[i, j].x:.2f}")
    else:
        print("No optimal solution found.")


# Create a model
model = gb.Model('bank_repay_optimization')
find_model_optimization(model)
